# Netflix Analysis using TV Shows Dataset from Kaggle

In [ ]:
# Import libraries
%matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Import data
file = "netflix_titles.csv"
data = pd.read_csv(file)

# Since column names are rare, we rename columns
rename_data = data.rename(columns={"show_id": "Show ID", "type": "Type", "title": "Title", "director": "Director",
                            "cast": "Cast", "country": "Country", "date_added": "Date Added", "release_year": "Release Year",
                            "rating": "Rating", "duration": "Duration", "listed_in": "Genre", "description": "Description"})

In [ ]:
# There is missing some information so we delete these data
clean_data = rename_data.replace(np.NaN)

In [ ]:
# We separate the TV Shows data and visualize the information
shows_country = clean_data[clean_data["Type"] == 'TV Show']
shows_country

In [ ]:
# We visualize that the total of all data set is 6234 and after selecting the tv shows, we have 1969
print('Length of Total Dataset: ' + str(len(clean_data)))
print('Length of TV Shows Dataset: ' + str(len(shows_country)))

In [ ]:
# We count the number of shows per genre and save all the genres we have in our dataset
genre_list_show = [] # Shows
genre_count_show = {} # Shows counting

for listed in shows_country['Genre']:
    split_gen = [x.strip() for x in listed.split(',')] # Split in list where there is more than one value
    
    # Counting dictionary
    for i in range(len(split_gen)): 
        # Check if exists in genre_list or not and add it
        if split_gen[i] not in genre_list_show: 
            genre_list_show.append(split_gen[i]) 
        # Count    
        if split_gen[i] in genre_count_show:
            genre_count_show[split_gen[i]] = genre_count_show[split_gen[i]] + 1
        else:
            genre_count_show[split_gen[i]] = 1

In [ ]:
# Ploting the total counts
# Save to dataframe
df_genres_shows = pd.DataFrame.from_dict(genre_count_show, orient='index', columns=['Total Data'])
# Sort values
sorted_genres_show = df_genres_shows.sort_values(by=['Total Data'], ascending=False)
sorted_genres_show.head(10)
# Bar plot
x_genres_shows = np.arange(len(sorted_genres_show['Total Data']))
tick_gslocations = [value for value in x_genres_shows]
width = 0.55
#plt.figure(figsize=(20,10))
plt.bar(x_genres_shows, sorted_genres_show['Total Data'], width=width, color='indigo', alpha=0.5) 
plt.xticks(tick_gslocations, sorted_genres_show.index, rotation="vertical")
plt.autoscale(tight=True)
plt.xlabel("Genres")
plt.ylabel("Total data")
plt.title("Show Total Genres")
plt.tight_layout()
for i, v in enumerate(sorted_genres_show['Total Data']):
    plt.text(i, v, str(v), color='k', ha='center', fontsize=7)
    
plt.savefig("TotalGenresShow.png")

In [ ]:
# Now we count the shows per country and extract the countries that have shows in Netflix library
country_list_show = [] # Save countries
country_count_show = {} # Count shows per country

for listed_in in shows_country['Country']:
    split_country = [x.strip() for x in listed_in.split(',')] # Split in list if there are more than one collaboration
  
    # To save the data
    for j in range(len(split_country)): 
        # Check if exists in country_list or not and add it
        if split_country[j] not in country_list_show: 
            country_list_show.append(split_country[j]) 
        # Count genres per country
        if split_country[j] in country_count_show:
            country_count_show[split_country[j]] = country_count_show[split_country[j]] + 1
        else:
            country_count_show[split_country[j]] = 1

In [ ]:
# We go deeper into the information to analyze the number of shows separated by genre existing per country
# We create a matrix where columns are genre list and rows are the countries
matrizSh = np.zeros((len(country_list_show),len(genre_list_show)))
# Save values of countries and genres
ctempo = shows_country['Country'].values
gtempo = shows_country['Genre'].values

for i in range(len(ctempo)): # We are in a row
    #print(i)
    cstmp=ctempo[i].split(',') # Separate the data if there is more than one country
    #print(cstmp)
    for k in range(len(cstmp)): # Analyzing each country of the previous split, we are still in the same row
        gctmp=gtempo[i].split(',') # We jump to the next cell where there are the genres and separate them
        #print(gctmp)
        for j in range(len(gctmp)): # Here, we are going to move through each genre for one country 
            # We use the matrix as a xy coordinate system where x would be the index of countries, and y would be the columns of genre.
            # To get the index of each value, we use the list information of countries and genres and the function .index to look for the respective country and genre
            # We sum +1 when we get the xy coordinates
            try: 
                matrizSh[country_list_show.index(cstmp[k].strip()),genre_list_show.index(gctmp[j].strip())]=matrizSh[country_list_show.index(cstmp[k].strip()),genre_list_show.index(gctmp[j].strip())]+1
            except:
                print('Error') # Just to verify that we gather all the data

In [ ]:
# We convert the matrix information to a dataframe
df_shows = pd.DataFrame(data=matrizSh, index=country_list_show, columns=genre_list_show)
df_shows

In [ ]:
# We analyze the top 10 countries with more TV Shows
# To gather this information we sort the values
country_df_show = pd.DataFrame.from_dict(country_count_show, orient='index', columns=['Total Data'])
sorted_countries_show = country_df_show.sort_values(by=['Total Data'], ascending=False)
sorted_countries_show.head(10)

In [ ]:
# We plot the top 10 countries with their respective counting shows by genre
for a in range(10):
    select_country = sorted_countries_show.index[a] # Country
    country_genres = df_shows.loc[df_shows.index == select_country,:] # Data from that country
    # Dataframe to sort the values
    df_sorted_genre = pd.DataFrame(country_genres).sort_values(by=select_country, axis=1, ascending=False)
    # We transpose the row data to column data to make easier the ploting 
    df_sorted_genre_T = df_sorted_genre.T
    
    # Bar Plot
    width = 0.55
    ax_bar = df_sorted_genre_T.plot(kind='bar', facecolor='slateblue', width=width, legend=None)
    plt.title(select_country+' Shows per Genre')
    plt.ylabel('Number of Titles')
    plt.xlabel('Genres')
    plt.tight_layout()

    # To enumarate the values in bars
    for i, v in enumerate(df_sorted_genre_T[select_country]):
        ax_bar.text(i, v, str(v), color='k', ha='center', fontsize=6)
        
    # Save figures
    plt.savefig('TVShow_Country' + str(a) + '.png')

In [ ]:
# Export dataframes of TV Shows data set and TV Shows per genre per country
shows_country.to_csv("Netflix_TVShows_Data.csv", index=True, header=True)
df_shows.to_csv('Netflix_TVShows_perCountry.csv', index=True, header=True)
